In [1]:
from __future__ import division, print_function
%matplotlib inline
from importlib import reload  # Python 3
import utils; reload(utils)
from utils import *
from keras.layers.merge import dot, add, concatenate

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)
Using TensorFlow backend.


In [2]:
#path = "data/ml-latest-small/"  # from https://grouplens.org/datasets/movielens/
#path = "data/ml-20m/"
path = "datas/movielens/" #from https://github.com/ChicagoBoothML/DATA___MovieLens___20M
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

batch_size=64
#batch_size=1

In [3]:
#20M dataset
#ratings = pd.read_csv(path+'ratings.csv')

#2M dataset
#ratings = pd.read_csv(path+'ratings01.csv')

#100k dataset
ratings = pd.read_csv(path+'ratings100k.csv')

In [4]:
#show 5 first file rows
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [5]:
#nb of lines
len(ratings)

100000

In [44]:
ratings.rating.mean()

3.5076049999999999

## datas prep
1. read files : ratings + movies
1. get list of unique items in ratings : users ID & movies ID
1. reorder users ID & movies ID to have countinuous integer items

In [6]:
#read film names
movie_names = pd.read_csv(path+'movies.csv').set_index('movieId')['title'].to_dict

#https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_dict.html
#https://stackoverflow.com/questions/30060616/python-pandas-to-dict-function

In [7]:
movie_names1 = pd.read_csv(path+'movies.csv').set_index('movieId')['title']
print(movie_names1[:5])

movieId
1                      Toy Story (1995)
2                        Jumanji (1995)
3               Grumpier Old Men (1995)
4              Waiting to Exhale (1995)
5    Father of the Bride Part II (1995)
Name: title, dtype: object


In [8]:
movie_names2 = pd.read_csv(path+'movies.csv').set_index('movieId')
print(movie_names2[:5])

                                      title  \
movieId                                       
1                          Toy Story (1995)   
2                            Jumanji (1995)   
3                   Grumpier Old Men (1995)   
4                  Waiting to Exhale (1995)   
5        Father of the Bride Part II (1995)   

                                              genres  
movieId                                               
1        Adventure|Animation|Children|Comedy|Fantasy  
2                         Adventure|Children|Fantasy  
3                                     Comedy|Romance  
4                               Comedy|Drama|Romance  
5                                             Comedy  


In [9]:
usersUniq = ratings.userId.unique()
moviesUniq = ratings.movieId.unique()

print(usersUniq[:20])
print(moviesUniq[:20])

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[  2  29  32  47  50 112 151 223 253 260 293 296 318 337 367 541 589 593 653 919]


In [10]:
# userId and movieId become ditionary elements with values ranging from 0 to max len 
userUniqID2idx = {userID:index for index,userID in enumerate(usersUniq)}
movieUniqID2idx = {movieID:index for index,movieID in enumerate(moviesUniq)}

In [11]:
#correspondance nouvel id / ancien id
{movieID : movieUniqID2idx[movieID] for movieID in list(movieUniqID2idx)[:10]}

{1: 227,
 2: 0,
 3: 175,
 4: 1994,
 5: 1044,
 6: 364,
 7: 420,
 8: 1965,
 9: 1677,
 10: 365}

In [12]:
ratings.movieId[:20]

0       2
1      29
2      32
3      47
4      50
5     112
6     151
7     223
8     253
9     260
10    293
11    296
12    318
13    337
14    367
15    541
16    589
17    593
18    653
19    919
Name: movieId, dtype: int64

In [13]:
#We update the movie and user ids so that they are contiguous integers, which we want when using embeddings.
ratings.movieId = ratings.movieId.apply(lambda x: movieUniqID2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userUniqID2idx[x])

#https://chrisalbon.com/python/data_wrangling/pandas_apply_operations_to_dataframes/

In [14]:
ratings.movieId[:15]

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
Name: movieId, dtype: int64

In [15]:
ratings.movieId[200:220]

200    192
201    193
202    194
203    195
204    196
205    197
206    198
207    199
208    200
209    201
210     80
211    202
212    203
213    204
214    205
215    206
216    207
217    208
218    209
219    210
Name: movieId, dtype: int64

In [16]:
user_min, user_max, movie_min, movie_max = (ratings.userId.min(), 
    ratings.userId.max(), ratings.movieId.min(), ratings.movieId.max())

In [17]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()

In [18]:
user_min, user_max, movie_min, movie_max, n_users, n_movies

(0, 701, 0, 8226, 702, 8227)

### embeddings

In [19]:
#number of latent factors in each embedding
n_factors = 10

In [20]:
np.random.seed = 42

#https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.seed.html

In [21]:
#Randomly split into training and validation

#Create an array of the given shape and populate it with random samples from a uniform distribution over [0, 1).
msk = np.random.rand(len(ratings)) < 0.8
print(msk)

trn = ratings[msk]
val = ratings[~msk]

[ True False  True ..., False  True  True]


In [22]:
trn[:5]

,userId,movieId,rating,timestamp
0,0,0,3.5,1112486027
2,0,2,3.5,1112484819
3,0,3,3.5,1112484727
4,0,4,3.5,1112484580
5,0,5,3.5,1094785740


In [23]:
val[:5]

,userId,movieId,rating,timestamp
1,0,1,3.5,1112484676
7,0,7,4.0,1112485573
12,0,12,4.0,1112484798
13,0,13,3.5,1094785709
18,0,18,3.0,1094785691


### Dot product
The most basic model is a dot product of a movie embedding and a user embedding. Let's see how well that works:

latent factor = explicative variables
#### calculate the latent factors for each movie AND each user, so that we can predict ratings by a dot product of 2 vectors :
latent user factors vector * latent movie factors vector

In [24]:
user_min, user_max, movie_min, movie_max, n_users, n_movies, n_factors

(0, 701, 0, 8226, 702, 8227, 10)

In [25]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(input_dim=n_in, output_dim=n_out, input_length=1, embeddings_regularizer=l2(reg))(inp)

In [26]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

In [ ]:
u

In [ ]:
#model construction, no data yet (we love TF :D)
x = dot([u, m], axes=2)
x = Flatten()(x)
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')

In [ ]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=batch_size, epochs=1, 
          validation_data=([val.userId, val.movieId], val.rating))

In [ ]:
model.optimizer.lr=0.01

In [ ]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=batch_size, epochs=3, 
          validation_data=([val.userId, val.movieId], val.rating))

In [ ]:
model.optimizer.lr=0.001

In [ ]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=batch_size, epochs=4, 
          validation_data=([val.userId, val.movieId], val.rating))

In [ ]:
#best result at epoch 3 !

### add Bias
The problem is likely to be that we don't have bias terms - that is, a single bias for each user and each movie representing how positive or negative each user is, and how good each movie is. We can add that easily by simply creating an embedding with one output for each movie and each user, and adding it to our output.

In [27]:
def create_bias(inp, n_in):
    x = Embedding(input_dim=n_in, output_dim=1, input_length=1)(inp)
    return Flatten()(x)

In [28]:
ub = create_bias(user_in, n_users)
mb = create_bias(movie_in, n_movies)

In [29]:
x = dot([u, m], axes=2)
x = Flatten()(x)
x = add([x, ub])
x = add([x, mb])
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')

In [30]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=batch_size, epochs=1, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80070 samples, validate on 19930 samples
Epoch 1/1
80070/80070 [==============================] - 7s - loss: 10.2833 - val_loss: 5.8871


In [31]:
model.optimizer.lr=0.001

In [32]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=batch_size, epochs=10, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80070 samples, validate on 19930 samples
Epoch 1/10
80070/80070 [==============================] - 7s - loss: 3.7607 - val_loss: 2.7404
Epoch 2/10
80070/80070 [==============================] - 7s - loss: 2.2656 - val_loss: 2.1162
Epoch 3/10
80070/80070 [==============================] - 7s - loss: 1.8516 - val_loss: 1.8806
Epoch 4/10
80070/80070 [==============================] - 7s - loss: 1.6766 - val_loss: 1.7633
Epoch 5/10
80070/80070 [==============================] - 7s - loss: 1.5802 - val_loss: 1.6871
Epoch 6/10
80070/80070 [==============================] - 7s - loss: 1.5122 - val_loss: 1.6251
Epoch 7/10
80070/80070 [==============================] - 7s - loss: 1.4551 - val_loss: 1.5687
Epoch 8/10
80070/80070 [==============================] - 7s - loss: 1.4031 - val_loss: 1.5190
Epoch 9/10
80070/80070 [==============================] - 7s - loss: 1.3534 - val_loss: 1.4688
Epoch 10/10
80070/80070 [==============================] - 7s - loss: 1.3062 - val_loss: 1.4217

In [ ]:
model.save_weights(model_path+'bias.h5')

In [33]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=batch_size, epochs=5, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80070 samples, validate on 19930 samples
Epoch 1/5
80070/80070 [==============================] - 7s - loss: 1.2600 - val_loss: 1.3754
Epoch 2/5
80070/80070 [==============================] - 7s - loss: 1.2147 - val_loss: 1.3323
Epoch 3/5
80070/80070 [==============================] - 7s - loss: 1.1703 - val_loss: 1.2903
Epoch 4/5
80070/80070 [==============================] - 7s - loss: 1.1272 - val_loss: 1.2517
Epoch 5/5
80070/80070 [==============================] - 7s - loss: 1.0850 - val_loss: 1.2143


In [ ]:
model.save_weights(model_path+'bias.h5')

In [34]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=batch_size, epochs=5, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80070 samples, validate on 19930 samples
Epoch 1/5
80070/80070 [==============================] - 7s - loss: 1.0446 - val_loss: 1.1788
Epoch 2/5
80070/80070 [==============================] - 7s - loss: 1.0060 - val_loss: 1.1476
Epoch 3/5
80070/80070 [==============================] - 7s - loss: 0.9694 - val_loss: 1.1172
Epoch 4/5
80070/80070 [==============================] - 7s - loss: 0.9347 - val_loss: 1.0898
Epoch 5/5
80070/80070 [==============================] - 7s - loss: 0.9023 - val_loss: 1.0648


In [ ]:
model.save_weights(model_path+'bias.h5')

In [35]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=batch_size, epochs=5, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80070 samples, validate on 19930 samples
Epoch 1/5
80070/80070 [==============================] - 7s - loss: 0.8722 - val_loss: 1.0414
Epoch 2/5
80070/80070 [==============================] - 7s - loss: 0.8436 - val_loss: 1.0206
Epoch 3/5
80070/80070 [==============================] - 7s - loss: 0.8172 - val_loss: 1.0023
Epoch 4/5
80070/80070 [==============================] - 7s - loss: 0.7928 - val_loss: 0.9853
Epoch 5/5
80070/80070 [==============================] - 7s - loss: 0.7701 - val_loss: 0.9707


In [ ]:
model.save_weights(model_path+'bias.h5')

In [36]:
#df.loc[(df['column_name'] == some_value) & df['other_column'].isin(some_values)]
val.loc[(ratings['userId'] == 10) & (ratings['movieId'] < 500)]
#t.loc[(t['movieId'] < 500)] 

,userId,movieId,rating,timestamp
960,10,227,4.5,1230858821
983,10,11,3.5,1230858799
987,10,369,5.0,1230858804
990,10,372,4.0,1230858880
1002,10,15,4.5,1230787503
1006,10,404,4.5,1230858969
1011,10,407,4.5,1230858904
1044,10,30,5.0,1230787560
1049,10,264,1.5,1230853686
1060,10,52,4.0,1230784904


In [42]:
#predict how much user 'X' would rate movie 'X'
model.predict([np.array([10]), np.array([30])])

array([[ 4.3209]], dtype=float32)

In [43]:
model.predict([np.array([10]), np.array([264])])

array([[ 4.4437]], dtype=float32)

## analyse results
To make the analysis of the factors more interesting, we'll restrict it to the top 2000 most popular movies.

In [ ]:
g=ratings.groupby('movieId')['rating'].count()
topMovies=g.sort_values(ascending=False)[:2000]
topMovies = np.array(topMovies.index)

In [ ]:
g.head()

In [ ]:
topMovies

First, we'll look at the movie bias term. We create a 'model' - which in keras is simply a way of associating one or more inputs with one more more outputs, using the functional API. Here, our input is the movie id (a single id), and the output is the movie bias (a single float).

In [ ]:
get_movie_bias = Model(movie_in, mb)
get_movie_bias

In [ ]:
movie_bias = get_movie_bias.predict(topMovies)
movie_bias

In [ ]:
movie_ratings = [(b[0], movie_names()[moviesUniq[i]]) for i,b in zip(topMovies,movie_bias)]

In [ ]:
movie_ratings[:20]

    #Zip the 2 lists together into one list of (key,value) tuples: zipped  
    zipped = list(zip(list_keys, list_values))

    #Inspect the list using print()  
    print(zipped)

    #Build a dictionary with the zipped list: data  
    data = dict(zipped)

    #Build and inspect a DataFrame from the dictionary: df  
    df = pd.DataFrame(data)  
    print(df)  

    script output:  
    [('Country', ['United States', 'Soviet Union', 'United Kingdom']), ('Total', [1118, 473, 273])]  
              Country  Total  
    0   United States   1118  
    1    Soviet Union    473  
    2  United Kingdom    273  
    

Now we can look at the top and bottom rated movies. These ratings are corrected for different levels of reviewer sentiment, as well as different types of movies that different reviewers watch.

In [ ]:
#ascending sort over col 0 (movie bias)
sorted(movie_ratings, key=itemgetter(0))[:15]

In [ ]:
#descending sort ovel col 0
sorted(movie_ratings, key=itemgetter(0), reverse=True)[:15]

We can now do the same thing for the embeddings.

In [ ]:
#get_movie_bias = Model(movie_in, mb)
get_movie_emb = Model(movie_in, m)
get_movie_emb.predict([topMovies]).shape

numpy.squeeze(a, axis=None): Remove single-dimensional entries from the shape of an array.  

    >>> x = np.array([[[0], [1], [2]]])
    >>> x.shape
    (1, 3, 1)
    >>> np.squeeze(x).shape
    (3,)

In [ ]:
movie_emb = np.squeeze(get_movie_emb.predict([topMovies]))
movie_emb.shape

Because it's hard to interpret 50 embeddings, we use PCA to simplify them down to just 3 vectors.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
movie_pca = pca.fit(movie_emb.T).components_

In [ ]:
fac0 = movie_pca[0]

In [ ]:
movie_comp = [(f, movie_names()[moviesUniq[i]]) for f,i in zip(fac0, topMovies)]

In [ ]:
sorted(movie_comp, key=itemgetter(0), reverse=True)[:10]

In [ ]:
sorted(movie_comp, key=itemgetter(0))[:10]

In [ ]:
fac1 = movie_pca[1]
movie_comp = [(f, movie_names()[moviesUniq[i]]) for f,i in zip(fac1, topMovies)]

In [ ]:
sorted(movie_comp, key=itemgetter(0), reverse=True)[:10]

In [ ]:
sorted(movie_comp, key=itemgetter(0))[:10]

In [ ]:
fac2 = movie_pca[2]
movie_comp = [(f, movie_names()[moviesUniq[i]]) for f,i in zip(fac2, topMovies)]

In [ ]:
sorted(movie_comp, key=itemgetter(0), reverse=True)[:10]

In [ ]:
sorted(movie_comp, key=itemgetter(0))[:10]

In [ ]:
#We can draw a picture to see how various movies appear on the map of these components. 

In [ ]:
#This picture shows the 1st and 3rd components.
# 'violent vs happy' & ???
start=50; end=100
X = fac1[start:end]
Y = fac2[start:end]
plt.figure(figsize=(15,15))
plt.scatter(X, Y)
for i, x, y in zip(topMovies[start:end], X, Y):
    plt.text(x,y,movie_names()[moviesUniq[i]], color=np.random.rand(3)*0.7, fontsize=14)
plt.show()

### Neural net

Rather than creating a special purpose architecture (like our dot-product with bias earlier), it's often both easier and more accurate to use a standard neural network. Let's try it! Here, we simply concatenate the user and movie embeddings into a single vector, which we feed into the neural net.

In [ ]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

In [ ]:
x = concatenate([u, m], axis=2)
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(70, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)
nn = Model([user_in, movie_in], x)
nn.compile(Adam(0.001), loss='mse')

In [ ]:
nn.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, epochs=8, 
          validation_data=([val.userId, val.movieId], val.rating))

In [ ]:
val.loc[(ratings['userId'] == 10) & (t['movieId'] < 500)]

In [ ]:
nn.predict([np.array([10]), np.array([12])])

In [ ]:
nn.predict([np.array([10]), np.array([36])])

In [ ]:
get_movie_emb2 = Model(movie_in, m)
movie_emb2 = np.squeeze(get_movie_emb2.predict([topMovies]))
movie_emb2.shape

In [ ]:
pca2 = PCA(n_components=3)
movie_pca2 = pca2.fit(movie_emb2.T).components_

In [ ]:
fac20 = movie_pca2[0]
movie_comp2 = [(f, movie_names()[moviesUniq[i]]) for f,i in zip(fac20, topMovies)]

In [ ]:
sorted(movie_comp2, key=itemgetter(0), reverse=True)[:10]

In [ ]:
sorted(movie_comp2, key=itemgetter(0))[:10]

In [ ]:
fac21 = movie_pca2[1]
movie_comp2 = [(f, movie_names()[moviesUniq[i]]) for f,i in zip(fac21, topMovies)]

In [ ]:
sorted(movie_comp2, key=itemgetter(0), reverse=True)[:10]

In [ ]:
sorted(movie_comp2, key=itemgetter(0))[:10]

In [ ]:
fac22 = movie_pca2[2]
movie_comp2 = [(f, movie_names()[moviesUniq[i]]) for f,i in zip(fac22, topMovies)]

In [ ]:
sorted(movie_comp2, key=itemgetter(0), reverse=True)[:10]

In [ ]:
sorted(movie_comp2, key=itemgetter(0))[:10]

In [ ]:
start=50; end=100
X = fac20[start:end]
Y = fac21[start:end]
plt.figure(figsize=(15,15))
plt.scatter(X, Y)
for i, x, y in zip(topMovies[start:end], X, Y):
    plt.text(x,y,movie_names()[moviesUniq[i]], color=np.random.rand(3)*0.7, fontsize=14)
plt.show()

In [ ]:
start=50; end=100
X = fac20[start:end]
Y = fac22[start:end]
plt.figure(figsize=(15,15))
plt.scatter(X, Y)
for i, x, y in zip(topMovies[start:end], X, Y):
    plt.text(x,y,movie_names()[moviesUniq[i]], color=np.random.rand(3)*0.7, fontsize=14)
plt.show()

In [ ]:
start=50; end=100
X = fac21[start:end]
Y = fac22[start:end]
plt.figure(figsize=(15,15))
plt.scatter(X, Y)
for i, x, y in zip(topMovies[start:end], X, Y):
    plt.text(x,y,movie_names()[moviesUniq[i]], color=np.random.rand(3)*0.7, fontsize=14)
plt.show()